In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os, sys
lib_path = os.path.abspath(os.path.join('../classifier/'))
sys.path.append(lib_path)
import classifier

In [ ]:
import nltk
import torch
import torchtext.vocab as vocab
import torch.optim as optim
import torch.autograd as autograd
import torch.nn as nn
import torch.utils.data as torchdata

In [5]:
import pickle

In [6]:
glove = vocab.GloVe(name='6B', dim=50)

# preprocess data

In [7]:
subset_train = 1

In [8]:
datapath = '../data/'

In [9]:
train = pd.read_csv(datapath + 'train.csv')
test = pd.read_csv(datapath + 'test.csv')
sub = pd.read_csv(datapath + 'sample_submission.csv')
train = train.iloc[:int(len(train) * subset_train), :]
test = test.iloc[:int(len(test) * subset_train), :]
test = test.fillna('unknown')
comment_types = train.columns[2:].values

In [ ]:
lang = classifier.Language()

In [ ]:
lang.process_sentences(train.comment_text.values,
                      test_sentences=list(test.comment_text.values))

In [ ]:
pickle.dump(lang, open(datapath + 'lang.pk', 'wb'))

In [10]:
lang = pickle.load(open(datapath + 'lang.pk', 'rb'))

In [11]:
init_embedding = classifier.create_init_embedding(lang, glove)

In [12]:
model = classifier.CharLSTM(lang.vocab_size,
                            char_size=127,
                           embedding_dim=50,
                           hidden_dim=128,
                            linear_dim=64, 
                           n_layers=1,
                           init_embedding=init_embedding,
                            char_embedding_dim=15,
                            char_hidden_dim = 32, 
                            char_n_layers=1,
                            dropout=.1,
                            gpu=True,
                           bidirectional=True,
                           char_bidirectional=True)

In [13]:
loss = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)
#model = model.cuda()

In [ ]:
trainer = classifier.CharTrainer(model,
                             optimizer,
                             loss,
                             lang.encoded_sentences,
                                 lang.encoded_chars,
                             train.loc[:, comment_types].values.astype(int),
                             batch_size =  600,
                            val_size=.01,
                                stratify = np.sum(train.loc[:, comment_types].values.astype(int), 1) )

In [ ]:
trainer.train(1)

In [14]:
results = np.zeros_like(sub.loc[:, comment_types].values)
n = 2 
for _ in range(n):
    print('************* model {} **********'.format(_))
    model = classifier.CharLSTM(lang.vocab_size,
                                char_size=127,
                               embedding_dim=50,
                               hidden_dim=128,
                                linear_dim=64, 
                               n_layers=1,
                               init_embedding=init_embedding,
                                char_embedding_dim=15,
                                char_hidden_dim = 32, 
                                char_n_layers=1,
                                dropout=.1,
                                gpu=True,
                               bidirectional=True,
                               char_bidirectional=True)
    loss = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0005)
    trainer = classifier.CharTrainer(model,
                                 optimizer,
                                 loss,
                                 lang.encoded_sentences,
                                     lang.encoded_chars,
                                 train.loc[:, comment_types].values.astype(int),
                                 batch_size =  128,
                                val_size=.01)
    trainer.train(4)
    final = classifier.make_submission_char(lang, model, sub, 'subm.csv', datapath,
                          comment_types)
    results += final

************* model 0 **********
Epoch: 0    loss: 0.1512,  val: 0.0599
Epoch: 1    loss: 0.1191,  val: 0.0541
Epoch: 2    loss: 0.1132,  val: 0.0509
Epoch: 3    loss: 0.1082,  val: 0.0492
predicting...
saving file...
done
************* model 1 **********
Epoch: 0    loss: 0.1560,  val: 0.0526
Epoch: 1    loss: 0.1193,  val: 0.0477
Epoch: 2    loss: 0.1137,  val: 0.0462
Epoch: 3    loss: 0.1094,  val: 0.0445
predicting...
saving file...
done


In [15]:
lang.encoded_sentences.shape

(95851, 50)

In [16]:
train.loc[:, comment_types].values.shape

(95851, 6)

In [17]:
results = results/n
sub.loc[:, comment_types] = results
sub.to_csv(datapath + 'ensemble.csv', index = False)

In [ ]:
classifier.make_submission_char(lang, model, sub, 'subm.csv', datapath,
                          comment_types)

In [ ]:
np.max(lang.encoded_test_sentences)

In [ ]:
np.max(lang.encoded_sentences)


In [ ]:
np.max(lang.encoded_chars)

In [ ]:
np.max(lang.encoded_test_chars)

In [ ]:
lang.vocab_size